### a.
*Exercise: Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.*

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import ssl
%load_ext tensorboard

In [2]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, 
                              kernel_initializer = keras.initializers.he_normal(), 
                              activation = tf.nn.elu)

In [4]:
model = keras.models.Sequential()
model.add(
    keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(RegularizedDense(100))



### b.
*Exercise: Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with `keras.datasets.cifar10.load_data()`. The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.*


In [5]:
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
assert not np.any(np.isnan(X_train_full))

In [8]:
model.add( keras.layers.Dense(10, activation= tf.nn.softmax))

In [9]:
model.compile(optimizer=keras.optimizers.SGD(lr = 5e-5),
              loss= keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"],
              )

In [10]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_model.h5" , save_best_only=True)

run_index = 1
run_logdir = os.path.join(os.curdir, "cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [11]:
%tensorboard --logdir=./cifar10_logs --port=6006


Reusing TensorBoard on port 6006 (pid 8724), started 0:02:37 ago. (Use '!kill 8724' to kill it.)

In [12]:
model.fit(X_train, y_train, epochs= 100, verbose = 1, 
          validation_data=(X_valid, y_valid), callbacks = callbacks, workers = 0)

mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
   1/1407 [..............................] - ETA: 0s - loss: 3.8969 - accuracy: 0.0312WARNING:tensorflow:From C:\Users\goofy\d2l\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1407/1407 [==============================] - 6s 4ms/step - loss: 2.2975 - accuracy: 0.2044 - val_loss: 2.0795 - val_accuracy: 0.2548
Epoch 2/100
1407/1407 [==============================] - 5s 4ms/step - loss: 2.0601 - accuracy: 0.2516 - val_loss: 2.0046 - val_accuracy: 0.2752
Epoch 3/100
1407/1407 [==============================] - 6s 4ms/step - loss: 2.0023 - accuracy: 0.2721 - val_loss: 1.9585 - val_accuracy: 0.2912
Epoch 4/100
1407/1407 [==============================] - 5s 4ms/step - loss: 1.9621 - accuracy: 0.2898 - val_loss: 1.9328 - val_accuracy: 0.3046
Epoch 5/100
1407/1407 [==============================] - 6